# Data analysis

Content table:
1. [Dependencies](#dependencies)
2. [Size analysis](#size-analysis)
3. [Analysis of overhanging trees](#analysis-of-overhanging-trees)
4. [Non-matching samples](#non-matching-samples)

### Dependencies

In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from tqdm import tqdm
import pickle
import seaborn as sn
from sklearn.preprocessing import normalize
import rasterio
from rasterio.mask import mask

### Size analysis

In [ ]:
# Load files
df_samples_meta = pd.read_csv("../data/test/dataset/samples_metadata.csv", sep=';')
print(len(df_samples_meta))
print(df_samples_meta.columns)
# categories of samples
categories = {
'b' : ['bare', 0],
'e' : ['extensive', 1],
'i' : ['intensive', 2],
'l' : ['lawn', 3],
's' : ['spontaneous', 4],
't' : ['terraces', 5],
}


In [ ]:
# histogram general of #samples per size
fig = plt.figure()
num_bins = 20
bins = ()
histo = plt.hist(np.log(df_samples_meta.area), log=False, bins=num_bins)
lst_xticks = np.exp(histo[1])
plt.xticks(histo[1], labels=np.exp(histo[1]).astype(int),rotation=90)
plt.show()

In [ ]:
# histogram per class of #samples per size
fig, axs = plt.subplots(2,3, figsize=(10,6),sharex= True)
for idx, cat in enumerate(categories.items()):
    ax = axs[idx // 3, idx % 3]
    ax.set_title(cat[1][0])
    ax.hist(np.log(df_samples_meta[df_samples_meta['class'] == cat[0]].area), log=False, bins=num_bins)
    ax.set_xticks(histo[1][::2], labels=np.exp(histo[1][::2]).astype(int),rotation=90)
# add a big axes, hide frame
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axes
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.ylabel('count [-]')
plt.xlabel('area [$m2$]', labelpad=30)
plt.suptitle("Histogram of the area per category")
plt.tight_layout()
fig.savefig("../results/data_analysis/size_histo_per_cat.png")
plt.show()

In [ ]:
# boxplot per class per class of #samples per size
lst_ticks_labels = [x[0] for x in categories.values()]
lst_ticks_labels.insert(0,"")
max_val = df_samples_meta.area.max()
fig = plt.figure()
plt.boxplot([np.log(df_samples_meta[df_samples_meta['class'] == cat[0]].area)
             for cat in categories.items()])
plt.xticks(ticks=range(len(categories)+1), labels=lst_ticks_labels)
plt.yticks(ticks=np.arange(0, np.log(max_val), np.log(max_val)/5),
            labels=np.exp(np.arange(0, np.log(max_val), np.log(max_val)/5)).astype(int))
plt.title('Boxplot of the area for each category')
plt.ylabel("area [$m^2$]")
plt.tight_layout()
fig.savefig("../results/data_analysis/size_boxplot_per_cat.png")
plt.show()

### Analysis of training results

#### repartition of size of mislabeled samples

In [ ]:
# load file
with open("../results/trainings/MSPP_num_epoch_50_290924/logs/samp_logs.pickle",'rb') as file:
    samp_logs = pickle.load(file)

best_epoch = 47
df_samp_logs_epoch = pd.DataFrame(samp_logs[best_epoch])
df_mislabeled = df_samp_logs_epoch[df_samp_logs_epoch.target != df_samp_logs_epoch.pred]

df_samp_logs_epoch_wo_rot = df_samp_logs_epoch[~df_samp_logs_epoch.egid.str.contains('_')]

df_mislabeled.to_csv('../test/mislabeled.csv', sep=';', index=False)
df_mislabeled = df_mislabeled[~df_mislabeled.egid.str.contains('_')]

df_mislabeled.to_csv('../test/mislabeled_raw.csv', sep=';', index=False)

df_samples_meta.EGID = df_samples_meta.EGID.astype(int)
df_mislabeled.egid = df_mislabeled.egid.astype(int)
df_samp_logs_epoch_wo_rot.egid = df_samp_logs_epoch_wo_rot.egid.astype(int)
areas_all = df_samples_meta[df_samples_meta.EGID.isin(df_samp_logs_epoch_wo_rot.egid.values.astype(int))][['EGID', 'area']].rename(str.lower, axis='columns')
areas_mislabeled = df_samples_meta[df_samples_meta.EGID.isin(df_mislabeled.egid.values.astype(int))][['EGID', 'area']].rename(str.lower, axis='columns')

df_mislabeled.set_index('egid', inplace=True)
df_samp_logs_epoch_wo_rot.set_index('egid', inplace=True)
areas_mislabeled.set_index('egid', inplace=True)
areas_all.set_index('egid',inplace=True)

df_mislabeled = df_mislabeled.join(areas_mislabeled, how='inner')
df_samp_logs_epoch_wo_rot = df_samp_logs_epoch_wo_rot.join(areas_all, how='inner')

print(f"Number of raw samples (without rotation) : {len(df_samp_logs_epoch_wo_rot)}")
print(f"Number of misclassified raw samples (without rotation) : {len(df_mislabeled)}")



In [ ]:
# histogram general of #samples per size
fig = plt.figure()
num_bins = 20
bins = ()
histo = plt.hist(np.log(df_mislabeled.area), log=False, bins=num_bins)
lst_xticks = np.exp(histo[1])
plt.xticks(histo[1], labels=np.exp(histo[1]).astype(int),rotation=90)
plt.ylabel('count [-]')
plt.xlabel('area [$m2$]', labelpad=30)
plt.suptitle("Histogram of the size for mislabeled samples.")
plt.tight_layout()
fig.savefig("../results/data_analysis/size_histo_misclassified.png")
plt.show()

In [ ]:
# general statistics

# test correlation
df_samp_logs_epoch_wo_rot.columns
df_matching = df_samp_logs_epoch_wo_rot.pred == df_samp_logs_epoch_wo_rot.target
df_corr_area_match = df_samp_logs_epoch_wo_rot.assign(match=df_matching.values)
print(f"\nThe correlation between the area and the matching results is : {round(df_corr_area_match.area.corr(df_corr_area_match.match),3)}")

# box plots
fig = plt.figure()
max_val = df_samp_logs_epoch_wo_rot.area.max()
plt.boxplot([np.log(x) for x in [df_samp_logs_epoch_wo_rot.area, df_mislabeled.area]])
plt.xticks(ticks=[1,2], labels=["dataset", 'misclassified'])
plt.yticks(ticks = np.arange(0, np.log(max_val), np.log(max_val)/5),
           labels=np.exp(np.arange(0, np.log(max_val), np.log(max_val)/5)).astype(int))
plt.title('Boxplot of the area for dataset and misclassified ($\\rho =-0.087$)')
plt.ylabel("area [$m^2$]")
plt.tight_layout()
fig.savefig("../results/data_analysis/size_boxplot_full_vs_mis.png")

### Analysis of overhanging trees

In [ ]:
src_overhanging = '../data/sources/gt_MNC_filtered.gpkg'
src_original = '../data/sources/gt_tot.gpkg'
src_rasters = "../data/sources/scratch_dataset"
original_roofs = gpd.read_file(src_original).to_crs(2056)
overhanging_roofs = gpd.read_file(src_overhanging).to_crs(2056)
print(f"represented classes : {overhanging_roofs['class'].unique()}")
print(f"columns of overhanging: {overhanging_roofs.columns}")
print(f"columns of original: {original_roofs.columns}")
print(f"Number of samples: {len(overhanging_roofs)}")
print(f"Difference with original : {len(original_roofs) - len(overhanging_roofs)} samples")

In [ ]:
# histogram of class of removed samples
removed_samples = original_roofs.merge(overhanging_roofs, how='left', on=['EGID', 'class'], indicator=True)
removed_samples = removed_samples[removed_samples['_merge'] == 'left_only'].drop(['_merge'], axis=1).reset_index(drop=True)
print(f"Number of samples : {len(removed_samples)}")
print("Removed samples in classes:")
print(removed_samples[['EGID','class']].groupby('class').count())

#### Filtering bares on NDVI values

In [3]:
def ndvi_samp_gen(arr_input):
    R = arr_input[1,:,:].astype(float)
    NIR = arr_input[0,:,:].astype(float)
    ndvi = np.divide(NIR - R, NIR + R, out=np.zeros(R.shape, dtype=float), where= NIR + R != 0)
    ndvi = ndvi.reshape((1,ndvi.shape[0], ndvi.shape[1]))
    return ndvi


In [ ]:
# get rasters
raster_list = []
for r, d, f in os.walk(src_rasters):
    for file in f:
        if file.endswith('.tif'):
            file_src = r + '/' + file
            file_src = file_src.replace('\\','/')
            raster_list.append(rasterio.open(file_src))

list_egid_to_remove = []
overlapping_roofs = []
nonmatching_roofs = []
for roof in tqdm(overhanging_roofs[overhanging_roofs['class'] == 'b'].itertuples(index=True), total=len(overhanging_roofs[overhanging_roofs['class'] == 'b']), desc="Clipping"):
    geom = roof.geometry
    cat = roof[5]
    egid = str(int(roof.EGID))
    
    # loop over the rasters to find the one matching
    matching_rasters = []
    matching_images = []
    sample_formats = ""
    for raster in raster_list:
        # catch the error when polygon doesn't match raster and just continue to next raster
        try:
            out_image, out_transform = mask(raster, [geom], crop=True)
            sample_formats = out_image.dtype
        except ValueError:
            continue
        else:
            if np.max(out_image) == 0:  # test if image is not full black
                continue
            matching_rasters.append(raster)
            matching_images.append((out_image, out_transform))

    # test if polygon match with one and only one raster:    
    if len(matching_rasters) == 0:
        nonmatching_roofs.append(egid)
        continue
    else:
        img_size_max = np.sum(matching_images[0][0].shape)
        for img, transf in matching_images:
            if np.sum(img.shape) > img_size_max:
                img_size_max = np.sum(img.shape)
                out_image = img
                out_transform = transf
    ndvi_canal = ndvi_samp_gen(out_image)
    out_image = np.concatenate([out_image, ndvi_canal])

    if cat == 'b' and np.mean(ndvi_canal, where=ndvi_canal!=0.0) > 0.05:
        list_egid_to_remove.append(egid)

print(f"Number of 'bare' samples to remove based on NDVI > 0.05 : {len(list_egid_to_remove)}")

In [ ]:
print(list_egid_to_remove)

In [ ]:
lst_test = []
lst_test.append(np.random.rand(3,2,2))
lst_test.append(np.random.rand(3,3,3))
lst_test.append(np.random.rand(3,2,4))


### non-matching samples

In [ ]:
import csv
samples_src = "./nonmatchingsamples.csv"
roofs_src = "../data/sources/gt_MNC_filtered.gpkg"
dataset = "../data/sources/tlm_dataset"

lst_samples = pd.read_csv(samples_src, sep=';').columns
lst_samples = [samp.replace("'","") for samp in lst_samples]
lst_samples = [int(samp.replace(" ","")) for samp in lst_samples]
print(lst_samples)


In [ ]:
roofs = gpd.read_file(roofs_src)
non_matching_roofs = roofs.loc[roofs.EGID.isin(lst_samples)]

# save non_matching roofs infos 
non_matching_roofs[['EGID','class']].to_csv('../results/data_analysis/non_matching_roofs.csv', sep=';', index=None)
raster_list = []

for r, d, f in os.walk(dataset):
    for file in f:
        if file.endswith('.tif'):
            file_src = r + '/' + file
            file_src = file_src.replace('\\','/')
            raster_list.append(rasterio.open(file_src))
print(raster_list)
for roof in non_matching_roofs.itertuples():
    egid = roof.EGID
    geom = roof.geometry
    for raster in raster_list:
        # catch the error when polygon doesn't match raster and just continue to next raster
        try:
            out_image, out_transform = mask(raster, [geom], crop=True)
            sample_formats = out_image.dtype
        except ValueError:
            continue
        else:
            if roof[5] != 'b':
                out_image = out_image/np.max(out_image)*255
                fig = plt.figure()
                plt.imshow(np.moveaxis(out_image[1::,...].astype(int),0,2))
                plt.title(f"{egid} - {roof[5]}")
                plt.show()

In [ ]:
print(non_matching_roofs[['class','EGID']].groupby('class').count())

In [ ]:
#print(non_matching_roofs.EGID.values)
print(roofs.loc[roofs.EGID.isin(non_matching_roofs.EGID.values),:])

matching_roofs = roofs.drop(roofs.loc[roofs.EGID.isin(non_matching_roofs.EGID.values),:].index, axis=0)
print(len(roofs))
print(len(non_matching_roofs))
print(len(matching_roofs))

matching_roofs.to_file('../data/sources/gt_matching_roofs.gpkg', driver='GPKG')
non_matching_roofs.to_file('../data/sources/gt_non_matching_roofs.gpkg', driver='GPKG')